## 5.1 Forecasting - Facebook Prophet
<font color=green> # Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.</font> [Source: Facebook GitHub](https://facebook.github.io/prophet/)

* [Prophet (FB Blog)-forecasting at scale](https://research.fb.com/blog/2017/02/prophet-forecasting-at-scale/)
* [Forecasting at Scale](https://peerj.com/preprints/3190.pdf)

In [ ]:
#importing required packages
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 10)
pd.set_option('display.max_rows', 500)
import plotly.graph_objects as go
#plot the graph using plolty
import plotly.offline as py
from fbprophet import Prophet #attention might have problems with holiday package
from fbprophet.plot import plot_cross_validation_metric
from fbprophet.plot import plot_plotly 
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
#The style package adds support for easy-to-switch plotting "styles" with 
## the same parameters as a matplotlib rc file (which is read at startup to configure matplotlib).
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
#Set a base path in such way that full execuation will be possible with one click
if os.path.split(os.getcwd())[-1]=='notebooks':
    os.chdir('C:/Users/dhame/ds_covid-19/')

'Your base path for this project is: '+os.path.split(os.getcwd())[-1]

## 5.2 Trivial Forecast (rolling mean)
* In statistics, a moving average (rolling average or running average) is a calculation to analyze data points by creating a series of averages of different subsets of the full data set. It is also called a moving mean (MM) or rolling mean and is a type of finite impulse response filter.(Source: Wiki)

In [ ]:
# generate an input df
df = pd.DataFrame({'X': np.arange(0,10)})
# take the window of 3 and write the average as y named column
df['y']=df.rolling(3).mean() 
df.head()

In [ ]:
# Import CSV file as dataframe
df_all = pd.read_csv('/data/processed/COVID_small_flat_table.csv',sep=';')
# select the data of only one country, here: germany 
df=df_all[['date','Germany']]
# rename the column name 
df=df.rename(columns={'date': 'ds','Germany': 'y'})
df.head()

In [ ]:
ax = df.set_index('ds').plot(figsize=(12, 8), logy=True)
ax.set_ylabel('Confimed cases per day')
ax.set_xlabel('Timeline in Days')
plt.show()

In [ ]:
# set the uncertainty interval to 95% (the Prophet default is 80%)
#my_model = Prophet(interval_width=0.95) # for piecwise linear model
my_model = Prophet(growth='logistic')   # for logistic model

In [ ]:
# the column 'cap' is only mandatory for the logistic model
df['cap']=1000000.
my_model.fit(df)

In [ ]:
# define the periods and the frequency 'D'== days
future_dates = my_model.make_future_dataframe(periods=7, freq='D')
future_dates['cap']=1000000. # only mandatory for the logistic model

In [ ]:
# predict according to the scikit-learn standard
forecast = my_model.predict(future_dates)
my_model.plot(forecast,uncertainty=True ); # fbprohet is also responisble for rendering the output

In [ ]:
fig = plot_plotly(my_model, forecast)  
fig.update_layout(width=1024,height=800,xaxis_title="Time",yaxis_title="Confirmed infected people from johns hopkins csse, log-scale)",)
fig.update_yaxes(type="log",range=[1.1,5.5])
py.iplot(fig)

In [ ]:
# sorting the values according to column named ds
forecast.sort_values(by='ds').head()

In [ ]:
#plot the graphs
my_model.plot_components(forecast);

In [ ]:
forecast[['ds','trend']].set_index('ds').plot(figsize=(12, 8),logy=True)

## 5.3 Cross-Validation
* Cross-validation, sometimes called rotation estimation or out-of-sample testing, is any of various similar model validation techniques for assessing how the results of a statistical analysis will generalize to an independent data set. <font color=red>(source:WiKi)
</font>

In [ ]:
df_cv = cross_validation(my_model, 
                         initial='40 days', # we take the first 30 days for training
                         period='1 days',  # every  days a new prediction run
                         horizon = '7 days') #we predict 7days into the future

In [ ]:
df_cv.sort_values(by=['cutoff','ds'])[0:12]
df_cv.head()

### 5.3.1 Performance matrix
* The Performance Matrix displays the performance of each Metric in the Campaign relative to the Variations that the users in each Variation Group see. It provides a variety of powerful tools to assist you in determining exactly how users are responding to each of your tested Variations, individually and in combination.

In [ ]:
df_p = performance_metrics(df_cv)
# the performance matrix shows the result for all horizon
df_p

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='mape',)

## 5.4 Diagonalplot 
+ Gives a good understanding for the under and over estimation w.r.t. magnitude 

In [ ]:
horizon='7 days'
df_cv['horizon']=df_cv.ds-df_cv.cutoff
date_vec=df_cv[df_cv['horizon']==horizon]['ds']
y_hat=df_cv[df_cv['horizon']==horizon]['yhat']
y=df_cv[df_cv['horizon']==horizon]['y']

In [ ]:
df_cv_7=df_cv[df_cv['horizon']==horizon]
type(df_cv['horizon'][0])

In [ ]:
# plotting the diagonal plot
fig, ax = plt.subplots(1, 1)
ax.plot(np.arange(max(y)),np.arange(max(y)),'--',label='diagonal')
ax.plot(y,y_hat,'-',label=horizon)  # horizon is a np.timedelta objct

ax.set_title('Diagonal Plot of forecasting')
ax.set_ylim(10, max(y))
ax.set_xlabel('Truth: y')
ax.set_ylabel('Prediciton: y_hat')
ax.set_yscale('log')
ax.set_xlim(10, max(y))
ax.set_xscale('log')
ax.legend(loc='best',prop={'size': 16});

## 5.5 Trivial Forecast
* let's create a Example (using data of germany)of trivial forecast to predicit 7 days into the future

In [ ]:
#defining function for mean absolute percentage error
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
parse_dates=['date']
df_all = pd.read_csv('data/processed/COVID_small_flat_table.csv',sep=';',parse_dates=parse_dates)
df_trivial=df_all[['date','Germany']]
df_trivial=df_trivial.rename(columns={'date': 'ds','Germany': 'y'})

### 5.5.1 One of the standard forecast: rolling mean 
+ In statistics, a moving average is a calculation to analyze data points by creating a series of averages of different subsets of the full data set.

### 5.5.2 An other standard forecast: exponentially-weighted moving average
+ An exponential moving average (EMA), also known as an exponentially weighted moving average (EWMA), is a first-order infinite impulse response filter that applies weighting factors which decrease exponentially. The weighting for each older datum decreases exponentially, never reaching zero.

In [ ]:
df_trivial['y_mean_r3']=df_trivial.y.rolling(3).mean() # take the average of 3 days

In [ ]:
# the result has to be shifted according to the prediciton horizon (here 7 days)
df_trivial['cutoff']=df_trivial['ds'].shift(7)
df_trivial['y_hat']=df_trivial['y_mean_r3'].shift(7)
df_trivial['horizon']=df_trivial['ds']-df_trivial['cutoff']
print('MAPE: '+str(MAPE(df_trivial['y_hat'].iloc[12:,], df_trivial['y'].iloc[12:,])))
df_trivial